In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_samples, silhouette_score, calinski_harabasz_score
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from itertools import product

pd.set_option('display.max_rows', 15)
np.random.seed(5)

sns.set()

In [2]:
from google.colab import drive
drive.mount('/content/drive')
batter = pd.read_csv('/content/drive/MyDrive/Diss/Code/CSV/batter_for_clustering.csv')

Mounted at /content/drive


In [3]:
batter_for_clustering = batter.copy().drop(['name','country','prelim_metric','pre-rank'],axis=1)
batter_for_clustering

,average,strike_rate,boundary_per_ball,boundary_index,finishing_index,runs_without_boundary_index,big_match_index
0,32.48,139.55,0.189,3.829,0.128,10.350,0.291
1,32.66,136.71,0.187,4.185,0.065,10.750,0.222
2,51.50,137.67,0.163,4.382,0.281,17.438,0.337
3,29.84,134.69,0.201,4.109,0.079,9.149,0.218
4,34.69,144.87,0.204,4.337,0.112,10.910,0.225
...,...,...,...,...,...,...,...
1297,3.00,48.00,0.080,0.500,0.000,1.000,0.000
1298,2.75,29.72,0.054,0.500,0.000,0.750,0.000
1299,2.75,27.50,0.025,0.250,0.000,1.750,0.000
1300,2.25,32.14,0.000,0.000,0.000,2.250,0.000


In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
batter_for_clustering_scaled= scaler.fit_transform(batter_for_clustering)

In [5]:
batter_for_clustering_scaled

array([[ 1.30610588,  1.07729054,  1.1284799 , ..., -0.43954647,
         0.98167219,  2.49798607],
       [ 1.32361762,  0.97677539,  1.09370896, ..., -0.77585764,
         1.09657193,  1.76910627],
       [ 3.15651345,  1.01075234,  0.67645768, ...,  0.37720923,
         3.0176955 ,  2.98390594],
       ...,
       [-1.58625044, -2.88845682, -1.72273715, ..., -1.12284536,
        -1.48867213, -0.57598526],
       [-1.63489417, -2.72423488, -2.1573739 , ..., -1.12284536,
        -1.34504746, -0.57598526],
       [-1.65921604, -2.88562541, -2.1573739 , ..., -1.12284536,
        -1.41685979, -0.57598526]])

In [6]:
!pip install minisom
from minisom import MiniSom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for minisom: filename=MiniSom-2.3.0-py3-none-any.whl size=9018 sha256=7457046ba70e834c6aa8649e5231c9eb6778f14652a1080ab845c99bb21ea378
  Stored in directory: /root/.cache/pip/wheels/d4/ca/4a/488772b0399fec45ff53132ed14c948dec4b30deee3a532f80
Successfully built minisom


In [7]:
som = MiniSom(x=4,y=1,input_len=7,sigma=1.0,learning_rate=0.5)
som.random_weights_init(batter_for_clustering_scaled)

/usr/local/lib/python3.7/dist-packages/minisom.py:160: UserWarning: Warning: sigma is too high for the dimension of the map.
  warn('Warning: sigma is too high for the dimension of the map.')


In [8]:
som.train_random(data = batter_for_clustering_scaled, num_iteration = 100)

In [9]:
weights = som.get_weights()
weights

array([[[ 0.28506233,  0.4240836 ,  0.26306862,  0.27920554,
         -0.20287017,  0.46469075,  0.19000427]],

       [[ 0.7742756 ,  0.49995772,  0.48086308,  0.89237662,
         -0.34144753,  0.9294924 ,  0.89216822]],

       [[ 1.85853117,  0.77762679,  0.65424586,  1.65461642,
         -0.10650185,  1.91868168,  1.96615382]],

       [[ 1.26469355,  1.07421037,  1.32110201,  1.91318115,
         -0.538152  ,  0.82728055,  1.72705583]]])

In [10]:
pred = []
for i in batter_for_clustering_scaled:
  pred.append(som.winner(i)[0])

In [11]:
predictions_som = np.array(pred)
predictions_som

array([3, 3, 2, ..., 0, 0, 0])

In [12]:
print('Silhouette score :',silhouette_score(batter_for_clustering_scaled, predictions_som, metric='euclidean'))
print('Calinski-Harabasz Index :',metrics.calinski_harabasz_score(batter_for_clustering_scaled, predictions_som))
print('Davies-Bouldin Index :',metrics.davies_bouldin_score(batter_for_clustering_scaled, predictions_som))

Silhouette score : 0.196637471986873
Calinski-Harabasz Index : 282.02622604136826
Davies-Bouldin Index : 1.4160213359307705


In [13]:
columns = batter.drop(['name','country','prelim_metric','pre-rank'],axis=1).columns.tolist()

In [14]:
batter_for_clustering_scaled = pd.DataFrame(batter_for_clustering_scaled,columns = columns)
batter_for_clustering_scaled

,average,strike_rate,boundary_per_ball,boundary_index,finishing_index,runs_without_boundary_index,big_match_index
0,1.306106,1.077291,1.128480,1.723832,-0.439546,0.981672,2.497986
1,1.323618,0.976775,1.093709,2.016256,-0.775858,1.096572,1.769106
2,3.156513,1.010752,0.676458,2.178075,0.377209,3.017696,2.983906
3,1.049267,0.905282,1.337106,1.953828,-0.701122,0.636686,1.726852
4,1.521111,1.265579,1.389262,2.141111,-0.524959,1.142532,1.800797
...,...,...,...,...,...,...,...
1297,-1.561929,-2.162907,-0.766536,-1.010660,-1.122845,-1.704109,-0.575985
1298,-1.586250,-2.809885,-1.218559,-1.010660,-1.122845,-1.775921,-0.575985
1299,-1.586250,-2.888457,-1.722737,-1.216014,-1.122845,-1.488672,-0.575985
1300,-1.634894,-2.724235,-2.157374,-1.421368,-1.122845,-1.345047,-0.575985


In [15]:
batter_scaled = batter_for_clustering_scaled.copy()
batter_scaled['name'] = batter['name']
batter_scaled['country'] = batter['country']

cols = batter_scaled.columns.tolist()
cols = cols[-2:] + cols[:-2]


batter_scaled = batter_scaled [cols]
batter_scaled['class'] = predictions_som
batter_scaled = np.round(batter_scaled,3)

In [16]:
batter_scaled

,name,country,average,strike_rate,boundary_per_ball,boundary_index,finishing_index,runs_without_boundary_index,big_match_index,class
0,RG Sharma,INDIA,1.306,1.077,1.128,1.724,-0.440,0.982,2.498,3
1,MJ Guptill,NZ,1.324,0.977,1.094,2.016,-0.776,1.097,1.769,3
2,V Kohli,INDIA,3.157,1.011,0.676,2.178,0.377,3.018,2.984,2
3,PR Stirling,IRE,1.049,0.905,1.337,1.954,-0.701,0.637,1.727,3
4,AJ Finch,AUS,1.521,1.266,1.389,2.141,-0.525,1.143,1.801,3
...,...,...,...,...,...,...,...,...,...,...
1297,H Helva,TKY,-1.562,-2.163,-0.767,-1.011,-1.123,-1.704,-0.576,0
1298,Apon Mustafizur,SRB,-1.586,-2.810,-1.219,-1.011,-1.123,-1.776,-0.576,0
1299,W Uisuk,THAI,-1.586,-2.888,-1.723,-1.216,-1.123,-1.489,-0.576,0
1300,I Katzarski,BUL,-1.635,-2.724,-2.157,-1.421,-1.123,-1.345,-0.576,0


In [17]:
batter_scaled['class'].value_counts()

0    995
1    170
3     77
2     60
Name: class, dtype: int64

In [18]:
columns = batter_for_clustering_scaled.columns.tolist()
dict_0 = dict(zip(columns,np.round(weights[0][0],3)))
dict_1 = dict(zip(columns,np.round(weights[1][0],3)))
dict_2 = dict(zip(columns,np.round(weights[2][0],3)))
dict_3 = dict(zip(columns,np.round(weights[3][0],3)))

In [19]:
middle_ovr = pd.DataFrame()

In [20]:
for j in range(4):
  temp = batter_scaled[batter_scaled['class'] == j]
  temp['rating'] = 0
  diction = dict()
  diction = dict(zip(columns,np.round(weights[j][0],3)))
  print(diction)
  for i in columns: 
    temp['rating'] = np.round(temp['rating'] + (temp[i] * diction.get(i)),3)
  middle_ovr = middle_ovr.append(temp)

{'average': 0.285, 'strike_rate': 0.424, 'boundary_per_ball': 0.263, 'boundary_index': 0.279, 'finishing_index': -0.203, 'runs_without_boundary_index': 0.465, 'big_match_index': 0.19}
{'average': 0.774, 'strike_rate': 0.5, 'boundary_per_ball': 0.481, 'boundary_index': 0.892, 'finishing_index': -0.341, 'runs_without_boundary_index': 0.929, 'big_match_index': 0.892}
{'average': 1.859, 'strike_rate': 0.778, 'boundary_per_ball': 0.654, 'boundary_index': 1.655, 'finishing_index': -0.107, 'runs_without_boundary_index': 1.919, 'big_match_index': 1.966}
{'average': 1.265, 'strike_rate': 1.074, 'boundary_per_ball': 1.321, 'boundary_index': 1.913, 'finishing_index': -0.538, 'runs_without_boundary_index': 0.827, 'big_match_index': 1.727}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
middle_ovr.sort_values('rating',ascending=False)

,name,country,average,strike_rate,boundary_per_ball,boundary_index,finishing_index,runs_without_boundary_index,big_match_index,class,rating
343,Azhar Andani,PORT,4.859,1.595,1.042,5.355,-1.123,7.488,7.347,2,48.751
361,Zeeshan Kukikhel,HUN,4.543,2.344,3.284,8.230,-1.123,1.958,7.347,3,43.257
369,Saim Hussain,BUL,3.147,1.859,1.337,2.959,-0.231,3.227,6.470,2,32.005
478,M Spoors,CAN,4.437,1.008,1.424,3.343,1.012,1.915,5.762,2,30.391
138,Taranjeet Singh,ROM,3.278,2.595,3.302,4.974,-0.359,0.820,4.706,3,29.809
...,...,...,...,...,...,...,...,...,...,...,...
1236,Bilal Khan,OMA,-1.449,-1.755,-1.740,-1.373,2.331,-1.637,-0.576,0,-3.341
1229,Shapoor Zadran,AFG,-1.479,-2.392,-1.358,-1.232,1.343,-1.793,-0.576,0,-3.353
1259,Rubel Hossain,BAN,-1.465,-1.896,-1.671,-1.366,2.438,-1.685,-0.576,0,-3.430
1289,R Rampaul,WI,-1.124,-2.031,-2.157,-1.421,2.689,-1.376,-0.576,0,-3.439


In [22]:
temp

,name,country,average,strike_rate,boundary_per_ball,boundary_index,finishing_index,runs_without_boundary_index,big_match_index,class,rating
0,RG Sharma,INDIA,1.306,1.077,1.128,1.724,-0.440,0.982,2.498,3,12.960
1,MJ Guptill,NZ,1.324,0.977,1.094,2.016,-0.776,1.097,1.769,3,12.405
3,PR Stirling,IRE,1.049,0.905,1.337,1.954,-0.701,0.637,1.727,3,11.690
4,AJ Finch,AUS,1.521,1.266,1.389,2.141,-0.525,1.143,1.801,3,13.552
6,DA Warner,AUS,1.331,1.110,1.181,1.836,-0.514,1.142,2.181,3,12.936
...,...,...,...,...,...,...,...,...,...,...,...
715,AP Devcich,NZ,0.846,1.111,2.033,2.480,-1.123,0.235,2.065,3,14.057
737,Amir Zaib,PORT,0.724,1.737,1.998,1.864,-1.123,0.594,-0.576,3,9.087
762,Mohammed Nurji,MWI,3.059,2.197,1.963,0.879,2.080,-0.096,-0.576,3,8.312
772,Irfan Bhima,MWI,0.530,0.080,0.798,1.659,-1.123,0.450,2.065,3,9.526
